In [11]:
# READ
import numpy as np
import os
import scipy.io as sio
import gc


In [12]:


def all_file(user_dir):  # 获取指定路径下所有文件的全路径
    file_list = list()
    for root, dirs, files in os.walk(user_dir):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list


matFile = all_file(r"./eeg_raw_data/1")
for i in all_file(r"./eeg_raw_data/2"):
    matFile.append(i)
for i in all_file(r"./eeg_raw_data/3"):
    matFile.append(i)
matFile.sort()
data = []
for i in matFile:
    data.append(sio.loadmat(i))  # 读取该目录下的所有.mat文件
print(matFile)

['./eeg_raw_data/1/10_20151014.mat', './eeg_raw_data/1/11_20150916.mat', './eeg_raw_data/1/12_20150725.mat', './eeg_raw_data/1/13_20151115.mat', './eeg_raw_data/1/14_20151205.mat', './eeg_raw_data/1/15_20150508.mat', './eeg_raw_data/1/1_20160518.mat', './eeg_raw_data/1/2_20150915.mat', './eeg_raw_data/1/3_20150919.mat', './eeg_raw_data/1/4_20151111.mat', './eeg_raw_data/1/5_20160406.mat', './eeg_raw_data/1/6_20150507.mat', './eeg_raw_data/1/7_20150715.mat', './eeg_raw_data/1/8_20151103.mat', './eeg_raw_data/1/9_20151028.mat', './eeg_raw_data/2/10_20151021.mat', './eeg_raw_data/2/11_20150921.mat', './eeg_raw_data/2/12_20150804.mat', './eeg_raw_data/2/13_20151125.mat', './eeg_raw_data/2/14_20151208.mat', './eeg_raw_data/2/15_20150514.mat', './eeg_raw_data/2/1_20161125.mat', './eeg_raw_data/2/2_20150920.mat', './eeg_raw_data/2/3_20151018.mat', './eeg_raw_data/2/4_20151118.mat', './eeg_raw_data/2/5_20160413.mat', './eeg_raw_data/2/6_20150511.mat', './eeg_raw_data/2/7_20150717.mat', './eeg_

In [13]:
print(len(data))

45


In [14]:
# 获取每个.mat中eeg信号变量的键的列表与每次实验eeg信号的长度
keys = []
eegLen = []
for i in data:
    temKeys = []
    temLen = []
    flag = 0
    for j in i:
        temKeys.append(j)
        if(flag >= 3):  # 第三项以后才是eeg的数据
            temLen.append(i[j].shape[1])
        flag += 1
    temKeys = temKeys[3:]  # 获取每个字典第三项后所有的键
    keys.append(temKeys)
    eegLen.append(temLen)
del temKeys, temLen
gc.collect()

16533

In [15]:
length = 800 # 每组的长度

# 在数据末尾补0，因为ndarray要求每一维的长度一定
Len = []
for i in eegLen:
    Len.append(max(i))
maxLen = max(Len)  # 找到最大的实验长度，作为ndarray容器的长度
maxLen = (maxLen // length) * length

In [16]:
dt = np.dtype(np.float32)

eeg = np.zeros([len(data), 24, 62, maxLen], dtype=dt)  # 容器的初始化
for (singleKeys, singleLen, i) in zip(keys, eegLen, range(len(data))):  # 遍历.mat
    for (key, j) in zip(singleKeys, range(24)):  # 遍历24个实验
        for k in range(62):  # 遍历通道
            if(maxLen - singleLen[j] <= 0):
                eeg[i][j][k] = data[i][key][k][:maxLen]
            else:    
                tem = np.zeros([maxLen - singleLen[j]])  # 对每个序列都补0，变成最大长度
                eeg[i][j][k] = np.append(data[i][key][k], tem)
del data, tem
gc.collect()

0

In [17]:
print(eeg.shape)

(45, 24, 62, 51200)


In [18]:
eeg = eeg.reshape(eeg.shape[0], eeg.shape[1],  eeg.shape[2], -1, length)
eeg = eeg.swapaxes(-2, -3)

In [19]:
print(eeg.shape)

(45, 24, 64, 62, 800)


In [20]:
part1 = eeg[:15]
part2 = eeg[15:30]
part3 = eeg[30:45]

In [21]:
def downSample(input, originLen, targetLen):
    '''
    input: 数据
    originLen: 原长度
    targetLen: 目标长度
    '''

    gap = originLen // (targetLen - 1)

    downSamp = np.zeros([input.shape[0], input.shape[1], input.shape[2], input.shape[3],
                         targetLen])
    for i in range(input.shape[0]):  #  .mat
        for j in range(input.shape[1]):  # trials
            for k in range(input.shape[2]): # segment
                for x in range(input.shape[3]): #channel
                    downSamp[i][j][k][x][0] = input[i][j][k][x][0]  # 0单独处理
                    for m in range(gap - 1, originLen + 1, gap):  # 间隔采样
                        downSamp[i][j][k][x][m // gap + 1] = input[i][j][k][x][m]
    return downSamp

In [22]:
part1 = downSample(part1, originLen = 800, targetLen = 25)

In [23]:
part2 = downSample(part2, originLen = 800, targetLen = 25)

In [24]:
part3 = downSample(part3, originLen = 800, targetLen = 25)

In [25]:
eeg = np.concatenate((part1, part2, part3), axis=0)

In [26]:
print(eeg.shape)

(45, 24, 64, 62, 25)


In [27]:
def delete_zero(eeg):
    zeroDataIndex = []
    for i in range(eeg.shape[0]):
        if(eeg[i][0][-1] == 0 and eeg[i][0][-2] == 0):  # 实验中所有数据的和为0的实验
            zeroDataIndex.append(i)  # 记录下索引
    eeg = np.delete(eeg, zeroDataIndex, axis=0)  # 删除为0的数据
    return eeg

In [28]:
for i in range(45):
    for j in range(24):
        temp = delete_zero(eeg[i][j])
        np.save("trials/EEG_temporal_" + str(i) + "_" + str(j), temp)

In [29]:
#eegShape = nSample * 62(channel) * 100(timeLen)
#MapShape = nSample * 100(freq) * (9*9)
nSample = eeg.shape[0]
timeRange = eeg.shape[2]
time_headMap = np.zeros([nSample, 9, 9, timeRange])

In [30]:
eeg_sub_list = []
for i in range(45):
    temp_list = []
    for j in range(24):
        temp = np.load("trials/EEG_temporal_" + str(i) + "_" + str(j) + '.npy')
        temp_list.append(temp)
    eeg_sub_list.append([i%15, temp_list])

In [31]:
eeg_sub_list = sorted(eeg_sub_list,key=(lambda x:x[0]))

In [32]:
eeg_list = eeg_sub_list

In [33]:
for i in range(45):
    eeg_list[i] = eeg_list[i][1]

In [34]:
eeg_sub_list = []
for i in range(15):
    eeg_sub_list.append(eeg_list[3*i:3*i + 3])

In [35]:
len(eeg_sub_list)

15

In [36]:
timeLength = 25

# 电极对应到二维矩阵中，可对应map.xlsx中的图看看
for i in range(15):
    for j in range(3):
        for k in range(24):
            nSample = eeg_sub_list[i][j][k].shape[0]
            eeg_headMap = np.zeros([nSample, 9, 9, timeLength])
            for x in range(nSample):  # 样本数循环
                # 0-1行
                eeg_headMap[x][0][3] = eeg_sub_list[i][j][k][x][0]
                eeg_headMap[x][0][4] = eeg_sub_list[i][j][k][x][1]
                eeg_headMap[x][0][5] = eeg_sub_list[i][j][k][x][2]
                eeg_headMap[x][1][3] = eeg_sub_list[i][j][k][x][3]
                eeg_headMap[x][1][5] = eeg_sub_list[i][j][k][x][4]

                # 2-5行
                for m in range(2, 7):
                    for n in range(9):
                        eeg_headMap[x][m][n] = eeg_sub_list[i][j][k][x][(m - 2) * 9 + n + 5]

                # 第7行
                for m in range(1, 8):
                    eeg_headMap[x][7][m] = eeg_sub_list[i][j][k][x][m + 49]

                # 第8行
                for m in range(2, 7):
                    eeg_headMap[x][8][m] = eeg_sub_list[i][j][k][x][m + 55]
            np.save("trials/temporal_subject_" + str(i) + "_section_" + str(j) + "_trial_temp" + str(k), eeg_headMap)

In [37]:
import matplotlib
import matplotlib.pyplot as plt
import random

freqFirst = eeg_headMap.swapaxes(-1, -2).swapaxes(-2, -3)

randTr = random.randint(1, eeg_headMap.shape[1])
randTime = random.randint(0, 24)

arr = freqFirst[2][0]

plt.matshow(arr, cmap='hot_r')
plt.colorbar()
plt.show()

del freqFirst

<Figure size 480x480 with 2 Axes>

In [38]:
# 设置新的边长
sideLen = 32

In [39]:
del eeg_headMap
gc.collect()

2309

In [40]:
from scipy import interpolate

for i in range(15):
    for j in range(3):
        for k in range(24):
            eeg_headMap = np.load("trials/temporal_subject_" + str(i) + "_section_" + str(j) + "_trial_temp" + str(k) + ".npy")
            eeg_timeFirst = eeg_headMap.swapaxes(-1, -2).swapaxes(-2, -3)
            eegMap_bigger = np.zeros([eeg_timeFirst.shape[0], 25, sideLen, sideLen])
            for m in range(eeg_timeFirst.shape[0]):  # 样本数循环
                for n in range(25):  # 频率循环
                    x = np.arange(9)
                    y = np.arange(9)
                    z = eeg_timeFirst[m][n]  # 取出每一个平面

                    f = interpolate.interp2d(x, y, z, kind='cubic')  # cubic为三次
                    xnew = np.linspace(0, 8, sideLen)
                    ynew = np.linspace(0, 8, sideLen)
                    eegMap_bigger[m][n] = f(xnew, ynew)
            np.save("trials/temporal_subject_" + str(i) + "_section_" + str(j) + "_trial_interpolated_" + str(k) + ".npy", eegMap_bigger)

In [ ]:
os.mkdir("final_raw")

In [41]:
# 标准化
#sample * time * 32 * 32
for i in range(15):
    for j in range(3):
        for k in range(24):
            eegMap = np.load("trials/temporal_subject_" + str(i) + "_section_" + str(j) + "_trial_interpolated_" + str(k) + ".npy")
            for m in range(eegMap.shape[0]):
                for n in range(eegMap.shape[1]):
                    eegMap[m][n] = eegMap[m][n] - eegMap[m][n].mean()
                        
            for m in range(eegMap.shape[0]):
                for n in range(eegMap.shape[1]):
                    eegMap[m][n] = eegMap[m][n] / eegMap[m][n].std()

            # 换轴
            eegMap = eegMap.swapaxes(1, 2).swapaxes(2, 3)
    
            #添加维度
            raw_shape = list(eegMap.shape)
            raw_shape.append(1)

            eegMap = eegMap.reshape(raw_shape)
    
            # 降低精度为单精度（float32）
            dt = np.dtype(np.float32)
            eegMap = np.array(eegMap, dtype=dt)
            np.save("final_raw/subject_" + str(i) + "_section_" + str(j) + "_trial_" + str(k) + ".npy", eegMap)